<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0728/09_2_(solution)FCN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FCN_implementation

## 주요 함수
- x32, x16, x8 : tensorflow.keras.layers.Upsampling2D
    - UpSampling2D(size, interpolation='bilinear')
- x2 : tensorflow.keras.layers.Conv2DTranspose
    - Conv2DTranspose(filters, size, strides, padding='same')


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [ ]:
def FCN(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    # Block 1
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    # Block 2
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Block 3
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    out_28x28 = layers.Conv2D(num_classes, 1, activation='softmax', padding='same')(pool3)

    # Block 4
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    out_14x14 = layers.Conv2D(num_classes, 1, activation='softmax', padding='same')(pool4)

    # Block 5
    conv5 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv5)
    conv5 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv5)
    pool5 = layers.MaxPooling2D(pool_size=(2, 2))(conv5)

    # Block 6
    conv6 = layers.Conv2D(4096, 1, activation='relu', padding='same')(pool5)
    conv6 = layers.Conv2D(4096, 1, activation='relu', padding='same')(conv6)
    out_7x7 = layers.Conv2D(num_classes, 1, activation='softmax', padding='same')(conv6)

    # upsampling x2 & add
    up1 = layers.Conv2DTranspose(num_classes, 4, 2, padding='same')(out_7x7)
    add_14x14 = layers.Add()([up1, out_14x14])

    up2 = layers.Conv2DTranspose(num_classes, 4, 2, padding='same')(add_14x14)
    add_28x28 = layers.Add()([up2, out_28x28])

    # Output Layer
    out_8s = layers.UpSampling2D(size=(8, 8), interpolation='bilinear')(add_28x28)
    out_16s = layers.UpSampling2D(size=(16, 16), interpolation='bilinear')(add_14x14)
    out_32s = layers.UpSampling2D(size=(32, 32), interpolation='bilinear')(out_7x7)

    # model
    model_8s = Model(inputs=inputs, outputs=out_8s)
    model_16s = Model(inputs=inputs, outputs=out_16s)
    model_32s = Model(inputs=inputs, outputs=out_32s)
    return model_8s, model_16s, model_32s

In [ ]:
model_8s, model_16s, model_32s = FCN(input_shape=(224,224, 3), num_classes=21)

In [ ]:
model_8s.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_361 (Conv2D)            (None, 224, 224, 64  1792        ['input_22[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_362 (Conv2D)            (None, 224, 224, 64  36928       ['conv2d_361[0][0]']             
                                )                                                          

In [ ]:
model_16s.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_307 (Conv2D)            (None, 224, 224, 64  1792        ['input_19[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_308 (Conv2D)            (None, 224, 224, 64  36928       ['conv2d_307[0][0]']             
                                )                                                           

In [ ]:
model_32s.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_19 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_307 (Conv2D)         (None, 224, 224, 64)      1792      
                                                                 
 conv2d_308 (Conv2D)         (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_85 (MaxPoolin  (None, 112, 112, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_309 (Conv2D)         (None, 112, 112, 128)     73856     
                                                                 
 conv2d_310 (Conv2D)         (None, 112, 112, 128)     147584    
                                                           